In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm


In [2]:
#read this csv file /Users/tommi/Desktop/Uni_documents/Data/dragon-portfolio/commodity.csv
commodity = pd.read_csv('/Users/tommi/Desktop/Uni_documents/Data/dragon-portfolio/commodity.csv')

#/Users/tommi/Desktop/Uni_documents/Data/dragon-portfolio/EHI640_returns10-02-2025.csv
benchmark_volatility = pd.read_csv('/Users/tommi/Desktop/Uni_documents/Data/dragon-portfolio/EHI640_returns10-02-2025.csv')

#/Users/tommi/Desktop/Uni_documents/Data/dragon-portfolio/equity.csv
equity = pd.read_csv('/Users/tommi/Desktop/Uni_documents/Data/dragon-portfolio/equity.csv')

#/Users/tommi/Desktop/Uni_documents/Data/dragon-portfolio/fixed_income.csv
fixed_income = pd.read_csv('/Users/tommi/Desktop/Uni_documents/Data/dragon-portfolio/fixed_income.csv')

#/Users/tommi/Desktop/Uni_documents/Data/dragon-portfolio/gold.csv
gold = pd.read_csv('/Users/tommi/Desktop/Uni_documents/Data/dragon-portfolio/gold.csv')

#/Users/tommi/Desktop/Uni_documents/Data/dragon-portfolio/volatility-daily.csv
volatility_daily = pd.read_csv('/Users/tommi/Desktop/Uni_documents/Data/dragon-portfolio/volatility-daily.csv')

#read this csv file /Users/tommi/Desktop/Uni_documents/Data/dragon-portfolio/volatility-monthly.csv
volatility_monthly = pd.read_csv('/Users/tommi/Desktop/Uni_documents/Data/dragon-portfolio/volatility-monthly.csv')




In [3]:
equity.tail()


,Dates,SPXT Index,SPY US Equity,IVV US Equity,VOO US Equity,URTH US Equity,ACWI US Equity
6284,03/02/2025,13171.21,597.77,600.68,549.70,159.37,120.21
6285,04/02/2025,13266.45,601.78,604.69,553.35,160.64,121.30
6286,05/02/2025,13318.43,604.22,607.17,555.62,161.55,121.89
6287,06/02/2025,13366.96,606.32,609.40,557.62,162.06,122.31
6288,07/02/2025,13241.34,600.77,603.80,552.20,160.60,121.33


In [10]:
import pandas as pd
import statsmodels.api as sm

# Assuming 'equity' is already your DataFrame
# 1. Filter out rows where there is missing data for the benchmark or any proxy
equity_filtered = equity.dropna(subset=['SPXT Index', 'SPY US Equity', 'IVV US Equity', 'VOO US Equity', 'URTH US Equity', 'ACWI US Equity'])

# 2. Calculate daily returns (percentage change)
equity_returns = equity_filtered[['SPXT Index', 'SPY US Equity', 'IVV US Equity', 'VOO US Equity', 'URTH US Equity', 'ACWI US Equity']].pct_change().dropna()

# 3. Assign dependent variable (benchmark) and independent variables (proxies)
y = equity_returns['SPXT Index']  # Benchmark returns
X = equity_returns.drop(columns=['SPXT Index'])  # Proxies returns

# 4. Initialize variables to track the best proxy
r2_scores = {}
best_r2 = -float('inf')  # Initialize with the lowest possible R² value
best_proxy = None
best_model = None

# 5. Loop through proxies to run regression
for column in X.columns:
    X_column = X[column]
    
    # Add constant (intercept) to the independent variable
    X_const = sm.add_constant(X_column)
    
    # Fit the regression model
    model = sm.OLS(y, X_const).fit()
    
    # Get R² value for the current model
    r2_scores[column] = model.rsquared
    
    # Track the best model with the highest R²
    if model.rsquared > best_r2:
        best_r2 = model.rsquared
        best_proxy = column
        best_model = model

# 6. Output the best proxy and its regression details
print(f"Best proxy: {best_proxy} with R²: {best_r2}")
print(best_model.summary())

# 7. Display the R² values for all proxies
for proxy, r2 in r2_scores.items():
    print(f"R² for {proxy}: {r2}")


Best proxy: VOO US Equity with R²: 0.9946635246344645
                            OLS Regression Results                            
Dep. Variable:             SPXT Index   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.995
Method:                 Least Squares   F-statistic:                 6.354e+05
Date:                Sat, 15 Feb 2025   Prob (F-statistic):               0.00
Time:                        17:58:10   Log-Likelihood:                 19702.
No. Observations:                3411   AIC:                        -3.940e+04
Df Residuals:                    3409   BIC:                        -3.939e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------